# Laboratorio 2 - Inteligencia Artificial 

#### Sebastian Juárez - 21471
Link al repo: https://github.com/SebasJuarez/CC3045/tree/Lab2

### Task 1 - Regresión Lineal

1. ¿Por qué el modelo de Naive Bayes se le considera “naive”?

La razon por la que se le considera "naive" es por que el algoritmo asume que rodas las caracteristicas son independientes entre sí, lo que casi nunca es cierto en muchos datos. Pero a pesar de esto, el algoritmo si es eficiente en la clasificacion de los textos y tambien en detectetar spam.

2. Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda
¿cómo funciona el truco del Kernel para este modelo? (Lo que se espera de esta pregunta es que puedan
explicar en sus propias palabras la fórmula a la que llegamos que debemos optimizar de SVM en clase)

La formulacion que se trata de optimizar es 1/2(||w||)^2 que puede llegar a maximizar el margen entre clases.
El truco del Kernel, segun entiendo, transforma los datos de una manera en la que se evita pasar por un proceso de "transformacion" y asi evitar hacer un calculo que se haria de otra manera, o en otros terminos, crea un atajo.

3. Investigue sobre Random Forest y responda
a. ¿Qué tipo de ensemble learning es este modelo?

En ese caso el Random Forest una un tipo llamado Bagging, que entrena varios arboles de decision con diferentes muestras de datos y al final combina las predicciones.

b. ¿Cuál es la idea general detrás de Random Forest?

La idea es que usando muchos arboles, separandolos por subconjuntos y caracteristicas, se promedie y que eso haga que se reduzca el sobreajuste.

c. ¿Por qué se busca baja correlación entre los árboles de Random Forest?

Si la correlacion es baja, esto significa que hay mejor generalizacion, lo que siginifica que todos cometan los mismos errores y eso reduce la varianza. Por esto es que tener baja la correlación es lo ideal en este metodo.

### Task 2 - Naive Bayes

In [1]:
import re
import random
import math
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


### Naive Bayes - Codigo desde cero

In [6]:
import re
import random
import math
from collections import defaultdict

# Función para limpiar texto
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Leer archivo y procesar dataset
def load_dataset(file_path, split_ratio=0.8, seed=42):
    random.seed(seed)
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            label, message = line.strip().split('\t', 1)
            data.append((label, clean_text(message)))
    
    random.shuffle(data)
    split_index = int(len(data) * split_ratio)
    return data[:split_index], data[split_index:]

# Entrenar modelo Naive Bayes
def train_naive_bayes(training_data):
    word_counts = {'spam': defaultdict(int), 'ham': defaultdict(int)}
    class_counts = {'spam': 0, 'ham': 0}
    vocab = set()
    
    for label, message in training_data:
        class_counts[label] += 1
        words = message.split()
        for word in words:
            word_counts[label][word] += 1
            vocab.add(word)
    
    return word_counts, class_counts, vocab

# Predecir la clase de un mensaje
def predict(message, word_counts, class_counts, vocab, alpha=1):
    message_words = message.split()
    total_messages = sum(class_counts.values())
    log_prob_spam = math.log(class_counts['spam'] / total_messages)
    log_prob_ham = math.log(class_counts['ham'] / total_messages)
    
    for word in message_words:
        log_prob_spam += math.log((word_counts['spam'].get(word, 0) + alpha) / (sum(word_counts['spam'].values()) + alpha * len(vocab)))
        log_prob_ham += math.log((word_counts['ham'].get(word, 0) + alpha) / (sum(word_counts['ham'].values()) + alpha * len(vocab)))
    
    return 'spam' if log_prob_spam > log_prob_ham else 'ham'

# Evaluar el modelo
def evaluate(test_data, word_counts, class_counts, vocab):
    correct = 0
    total = len(test_data)
    for label, message in test_data:
        prediction = predict(message, word_counts, class_counts, vocab)
        if prediction == label:
            correct += 1
    return correct / total

train_data, test_data = load_dataset('data/entrenamiento.txt')
word_counts, class_counts, vocab = train_naive_bayes(train_data)
accuracy = evaluate(test_data, word_counts, class_counts, vocab)

print(f'Accuracy: {accuracy:.4f}')

# Clasificación de nuevos mensajes
while True:
    msg = input("Ingrese un mensaje (o 'salir' para salir): ")
    if msg.lower() == 'salir':
        break
    msg_clean = clean_text(msg)
    prediction = predict(msg_clean, word_counts, class_counts, vocab)
    print(f'Clasificación: {prediction}')


Accuracy: 0.9739


### Naive Bayes - Codigo con librerias

In [4]:
# Función para limpiar texto
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Leer archivo y procesar dataset
def load_dataset(file_path):
    data = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            label, message = line.strip().split('\t', 1)
            data.append(clean_text(message))
            labels.append(label)
    return data, labels

data, labels = load_dataset('data/entrenamiento.txt')
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_counts, y_train)

y_pred = model.predict(X_test_counts)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy (sklearn): {accuracy:.4f}')

# Clasificación de nuevos mensajes
while True:
    msg = input("Ingrese un mensaje (o 'exit' para salir): ")
    if msg.lower() == 'exit':
        break
    msg_clean = clean_text(msg)
    msg_vector = vectorizer.transform([msg_clean])
    prediction = model.predict(msg_vector)[0]
    print(f'Clasificación: {prediction}')

Accuracy (sklearn): 0.9856


### SVM - Codigo desde cero

In [7]:
import numpy as np
import random

# Cargar dataset
def load_lol_dataset(file_path, split_ratio=0.8, seed=42):
    random.seed(seed)
    data = np.genfromtxt(file_path, delimiter=',', skip_header=1)
    np.random.shuffle(data)
    
    split_index = int(len(data) * split_ratio)
    return data[:split_index, :-1], data[:split_index, -1], data[split_index:, :-1], data[split_index:, -1]

def linear_kernel(x1, x2):
    return np.dot(x1, x2)

# Implementación de SVM desde cero
def train_svm(X, y, lr=0.001, lambda_param=0.01, epochs=1000):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)
    bias = 0
    
    for _ in range(epochs):
        for i, x_i in enumerate(X):
            condition = y[i] * (np.dot(x_i, weights) - bias) >= 1
            if condition:
                weights -= lr * (2 * lambda_param * weights)
            else:
                weights -= lr * (2 * lambda_param * weights - np.dot(x_i, y[i]))
                bias -= lr * y[i]
    
    return weights, bias

def predict_svm(X, weights, bias):
    approx = np.dot(X, weights) - bias
    return np.sign(approx)

# Evaluación del modelo
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

X_train, y_train, X_test, y_test = load_lol_dataset('data/high_diamond_ranked_10min.csv')
y_train = np.where(y_train == 0, -1, 1)  # Convertir 0s a -1 para SVM
y_test = np.where(y_test == 0, -1, 1)

weights, bias = train_svm(X_train, y_train)
y_pred = predict_svm(X_test, weights, bias)

print(f'Accuracy SVM (sin librerías): {accuracy(y_test, y_pred):.4f}')


Accuracy SVM (sin librerías): 1.0000


### SVM - Codigo con librerias

In [17]:
import numpy as np
import random
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargar dataset de League of Legends
def load_lol_dataset(file_path):
    data = np.genfromtxt(file_path, delimiter=',', skip_header=1)
    return data[:, :-1], data[:, -1]

X, y = load_lol_dataset('data/high_diamond_ranked_10min.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo SVM con sklearn
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy SVM (con sklearn): {accuracy:.4f}')


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

Se compararon los dos algoritmos. En el primer algoritmo pudimos ver que los dos tuvieron buenos resultados pero el que se hizo usando las librerias tuvo mejores resulatados, como se puede esperar de la libreria. En el segundo algoritmo, podemos ver que el realizado desde cero tuvo buenos resultados que alcanzararon los valores esperados, pero creo que ocurrieron problemas ya que en el primer algoritmo dio una respuesta de 1 en accuracy pero en el segundo no pudo procesarse seguramente por problemas del dataset que se debe arreglar